# MMAE 350 — HW3 Companion Notebook  
## Thomas Algorithm (Direct) and Gauss–Seidel (Iterative)

**Purpose:** This notebook is provided to **verify** your hand calculations for HW3 and to explore convergence behavior.  
Complete the required **hand work first**, then use this notebook to check your results.

---

### What you will do
1. Define the matrix **A** and right-hand side **b** from HW3  
2. Solve with:
   - (A) a **Thomas Algorithm** implementation (tridiagonal direct solver)
   - (B) a **Gauss–Seidel** iterative solver
3. Compare against `numpy.linalg.solve`
4. Plot **residual norm** vs iteration for Gauss–Seidel


In [1]:
import numpy as np
import matplotlib.pyplot as plt


## 1) Define the system  $\mathbf{A}\mathbf{x}=\mathbf{b}$

Enter the matrix and vector exactly as in the HW3 PDF.

For HW3 (as written), use:

\begin{equation}
\mathbf{A}=
\begin{bmatrix}
4 & -1 & 0 & 0 \\
-1 & 4 & -1 & 0 \\
0 & -1 & 4 & -1 \\
0 & 0 & -1 & 3
\end{bmatrix},\qquad
\mathbf{b}=
\begin{bmatrix}
15 \\ 10 \\ 10 \\ 10
\end{bmatrix}.
\end{equation}


In [2]:
# TODO: Confirm these match the assignment PDF
A = np.array([
    [ 4.0, -1.0,  0.0,  0.0],
    [-1.0,  4.0, -1.0,  0.0],
    [ 0.0, -1.0,  4.0, -1.0],
    [ 0.0,  0.0, -1.0,  3.0],
], dtype=float)

b = np.array([15.0, 10.0, 10.0, 10.0], dtype=float)

print("A =\n", A)
print("b =", b)
print("shape(A) =", A.shape)


A =
 [[ 4. -1.  0.  0.]
 [-1.  4. -1.  0.]
 [ 0. -1.  4. -1.]
 [ 0.  0. -1.  3.]]
b = [15. 10. 10. 10.]
shape(A) = (4, 4)


## 2) Reference solution (for checking)

This is *not* the Thomas Algorithm — it's a general-purpose dense solver.
Use it as a reference to confirm your other results.


In [3]:
x_ref = np.linalg.solve(A, b)
print("Reference solution (numpy.linalg.solve):")
print(x_ref)


Reference solution (numpy.linalg.solve):
[5. 5. 5. 5.]


## 3) Thomas Algorithm (Tridiagonal Direct Solver)

The Thomas Algorithm expects the three diagonals:

- `lower[i] = A[i+1, i]` (length \(n-1\))
- `main[i]  = A[i, i]`   (length \(n\))
- `upper[i] = A[i, i+1]` (length \(n-1\))

### Your task
- Extract the diagonals from `A`
- Solve using `thomas_solve(...)`

> If your hand work is correct, the Thomas solution should match the reference solution.


In [5]:
def extract_tridiagonals(A):
    # Extract lower, main, upper diagonals from a tridiagonal matrix A.
    A = np.asarray(A, dtype=float)
    lower = np.diag(A, k=-1).copy()
    main  = np.diag(A, k=0).copy()
    upper = np.diag(A, k=1).copy()
    return lower, main, upper

lower, main, upper = extract_tridiagonals(A)
print("lower =", lower)
print("main  =", main)
print("upper =", upper)

type(lower)

lower = [-1. -1. -1.]
main  = [4. 4. 4. 3.]
upper = [-1. -1. -1.]


numpy.ndarray

### Provided Thomas solver

This implementation modifies working copies of the diagonals (so your inputs remain unchanged).


In [ ]:
def thomas_solve(lower, main, upper, b):
    # Solve a tridiagonal system Ax=b using the Thomas algorithm.
    lower = np.asarray(lower, dtype=float)
    main  = np.asarray(main,  dtype=float)
    upper = np.asarray(upper, dtype=float)
    b     = np.asarray(b,     dtype=float)

    n = main.size
    if lower.size != n - 1 or upper.size != n - 1 or b.size != n:
        raise ValueError("Sizes must be: main(n), b(n), lower(n-1), upper(n-1).")

    c_prime = np.zeros(n-1, dtype=float)
    d_prime = np.zeros(n,   dtype=float)

    denom = main[0]
    if abs(denom) < 1e-15:
        raise ZeroDivisionError("Zero (or near-zero) pivot at row 0.")
    c_prime[0] = upper[0] / denom
    d_prime[0] = b[0] / denom

    for i in range(1, n-1):
        denom = main[i] - lower[i-1] * c_prime[i-1]
        if abs(denom) < 1e-15:
            raise ZeroDivisionError(f"Zero (or near-zero) pivot at row {i}.")
        c_prime[i] = upper[i] / denom
        d_prime[i] = (b[i] - lower[i-1] * d_prime[i-1]) / denom

    denom = main[n-1] - lower[n-2] * c_prime[n-2]
    if abs(denom) < 1e-15:
        raise ZeroDivisionError(f"Zero (or near-zero) pivot at row {n-1}.")
    d_prime[n-1] = (b[n-1] - lower[n-2] * d_prime[n-2]) / denom

    x = np.zeros(n, dtype=float)
    x[-1] = d_prime[-1]
    for i in range(n-2, -1, -1):
        x[i] = d_prime[i] - c_prime[i] * x[i+1]

    return x


In [ ]:
import numpy as np

def thomas_solve_simple(lower, main, upper, b):
    """
    Solve Ax = b for a tridiagonal matrix using the Thomas algorithm.

    Uses:
      m_i   = lower[i-1] / b'_{i-1}
      b'_i  = main[i] - m_i * upper[i-1]
      d'_i  = b[i] - m_i * d'_{i-1}

    Assumes:
      - lower has length n-1
      - main  has length n
      - upper has length n-1
      - no zero pivots
    """
    lower = np.asarray(lower, dtype=float)
    main  = np.asarray(main,  dtype=float)
    upper = np.asarray(upper, dtype=float)
    b     = np.asarray(b,     dtype=float)

    n = main.size

    # Allocate modified coefficients
    b_prime = np.zeros(n)
    d_prime = np.zeros(n)

    # --- Forward sweep ---
    b_prime[0] = main[0]
    d_prime[0] = b[0]

    for i in range(1, n):
        m = lower[i-1] / b_prime[i-1]
        b_prime[i] = main[i] - m * upper[i-1]
        d_prime[i] = b[i] - m * d_prime[i-1]

    # --- Back substitution ---
    x = np.zeros(n)
    x[-1] = d_prime[-1] / b_prime[-1]

    for i in range(n-2, -1, -1):
        x[i] = (d_prime[i] - upper[i] * x[i+1]) / b_prime[i]

    return x

In [ ]:
x_thomas = thomas_solve(lower, main, upper, b)
print("Thomas solution:")
print(x_thomas)

print("\nMax abs difference vs reference:", np.max(np.abs(x_thomas - x_ref)))


## 4) Gauss–Seidel Iteration

Gauss–Seidel produces a sequence \(x^{(0)}, x^{(1)}, \dots\)

Two common stopping checks:
- **Residual-based (preferred):**  \(\|r\|/\|b\| < \varepsilon\) where \(r=b-Ax\)
- **Update-based:** \(\|x^{(k+1)}-x^{(k)}\|/\|x^{(k+1)}\| < \varepsilon\)

### Your task
Fill in the TODOs in `gauss_seidel(...)`.


In [ ]:
def gauss_seidel(A, b, x0=None, tol=1e-10, max_iter=200, use_residual=True):
    # Solve Ax=b by Gauss-Seidel iteration.
    # Returns: x (final), res_hist (normalized residual history)
    A = np.asarray(A, dtype=float)
    b = np.asarray(b, dtype=float)
    n = b.size

    if x0 is None:
        x = np.zeros(n, dtype=float)
    else:
        x = np.asarray(x0, dtype=float).copy()

    bnorm = np.linalg.norm(b)
    if bnorm == 0.0:
        bnorm = 1.0

    res_hist = []

    for k in range(max_iter):
        x_old = x.copy()

        # --- One GS sweep ---
        for i in range(n):
            # TODO: compute sum_{j<i} a_ij x_j (new values)
            s1 = 0.0
            for j in range(i):
                s1 += A[i, j] * x[j]

            # TODO: compute sum_{j>i} a_ij x_j (old values)
            s2 = 0.0
            for j in range(i+1, n):
                s2 += A[i, j] * x_old[j]

            # TODO: update x[i]
            x[i] = (b[i] - s1 - s2) / A[i, i]

        # --- Convergence check ---
        r = b - A @ x
        res = np.linalg.norm(r) / bnorm
        res_hist.append(res)

        if use_residual:
            if res < tol:
                break
        else:
            denom = np.linalg.norm(x)
            if denom == 0.0:
                denom = 1.0
            upd = np.linalg.norm(x - x_old) / denom
            if upd < tol:
                break

    return x, np.array(res_hist)


In [ ]:
import numpy as np

def gauss_seidel_simple(A, b, x, num_sweeps):
    """
    Perform Gauss–Seidel iteration for Ax = b.

    Assumes:
      - A is n×n
      - b is length n
      - x is initial guess (length n)
      - num_sweeps is the number of GS iterations to perform
    """
    n = len(b)

    for k in range(num_sweeps):
        for i in range(n):

            s1 = 0.0
            for j in range(i):
                s1 = s1 + A[i, j] * x[j]   # new values

            s2 = 0.0
            for j in range(i+1, n):
                s2 = s2 + A[i, j] * x[j]   # old values

            x[i] = (b[i] - s1 - s2) / A[i, i]

    return x

In [ ]:
# Initial guess from HW: x^(0)=0
x0 = np.zeros_like(b)

x_gs, res_hist = gauss_seidel(A, b, x0=x0, tol=1e-12, max_iter=200, use_residual=True)

print("Gauss-Seidel solution:")
print(x_gs)
print("\nIterations:", len(res_hist))
print("Final normalized residual:", res_hist[-1])

print("\nMax abs difference vs reference:", np.max(np.abs(x_gs - x_ref)))


## 5) Plot: residual norm vs iteration

This plot helps you visualize convergence.


In [ ]:
plt.figure()
plt.semilogy(np.arange(1, len(res_hist)+1), res_hist, marker='o')
plt.xlabel("Iteration")
plt.ylabel(r"Normalized residual $\|r^{(k)}\|/\|b\|$")
plt.title("Gauss-Seidel Convergence")
plt.grid(True, which="both")
plt.show()


## 6) Optional: Compare first two sweeps to your hand work

If you computed \(x^{(1)}\) and \(x^{(2)}\) by hand, you can check them here.


In [ ]:
def gauss_seidel_sweeps(A, b, x0, num_sweeps=2):
    A = np.asarray(A, dtype=float)
    b = np.asarray(b, dtype=float)
    x = np.asarray(x0, dtype=float).copy()
    n = b.size
    xs = [x.copy()]

    for _ in range(num_sweeps):
        x_old = x.copy()
        for i in range(n):
            s1 = sum(A[i, j]*x[j] for j in range(i))
            s2 = sum(A[i, j]*x_old[j] for j in range(i+1, n))
            x[i] = (b[i] - s1 - s2) / A[i, i]
        xs.append(x.copy())
    return xs

xs = gauss_seidel_sweeps(A, b, x0=np.zeros_like(b), num_sweeps=2)
print("x^(0) =", xs[0])
print("x^(1) =", xs[1])
print("x^(2) =", xs[2])


## 7) Reflection (write this in your PDF submission)

In 1–2 sentences each:
- When would you prefer a **direct** method like Thomas?
- When would you prefer an **iterative** method like Gauss–Seidel?
